In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
# Adding system path
import sys, pathlib
sys.path.append(str(pathlib.Path.cwd().parent.parent))
# sys.path

In [ ]:
# Set to show warnings only once
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Setting up displays
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

import math
from plotly import tools
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from tabulate import tabulate
color_scheme=["red","blue","green","orange","purple","brown","pink","gray","olive","cyan","darkviolet","goldenrod","darkgreen","chocolate","lawngreen"]

In [ ]:
# Setting up autoreload for libs
%load_ext autoreload
%autoreload 2
%aimport optiml.backend.warehouse_profile

In [ ]:
# Initialize connection to Snowflake and set analysis date
from optiml.connection import SnowflakeConnConfig
connection = SnowflakeConnConfig(accountname='jg84276.us-central1.gcp',warehousename="XSMALL_WH").create_connection()
# Initialize local environment
import os
cache_dir = os.path.expanduser('~/data/kiva')
# Initialize query library
from optiml.backend.warehouse_profile import WarehouseProfile
wqlib =WarehouseProfile(connection, 'KIV', cache_dir)
sdate = '2022-09-12'
edate = '2022-10-12'
print(f"The analysis is carried our for date range {sdate} to {edate}")
wh_name="PROD_WH"
time_delta="hour"
sdate_load="2022-10-10"
edate_load="2022-10-11"
threshold=2

### Warehouse load analysis

In [ ]:
df=wqlib.wh_queued_load_ts(start_date=sdate,end_date=edate,wh_name=wh_name,delta=time_delta)
fig1=px.bar(df, x=df["hourly_start_time"], y=[df["avg_queued_load"], df["avg_running"]], title="PROD_WH", color_discrete_sequence=color_scheme)
fig1.show()



In [ ]:

df=wqlib.wh_queued_load_ts(start_date=sdate,end_date=edate,wh_name=wh_name,delta=time_delta)
trace1 = go.Bar(
    x=df["hourly_start_time"], y=df["avg_queued_load"],
    name='Queued Load',marker=dict(color='rgb(222,0,0)')
)
trace2 = go.Bar(
    x=df["hourly_start_time"], y=df["avg_running"],
    name='Running load',marker=dict(color='rgb(0,0,255)')
)
trace3 = go.Scatter(
  x=df['hourly_start_time']
  ,y=df['credits_used']
  ,fill='tozeroy'
  ,name='Credits consumed'
  )


fig=make_subplots(rows=1,cols=1,shared_yaxes=True)
fig.append_trace(trace1, 1,1)
fig.append_trace(trace2, 1, 1)
fig.append_trace(trace3,1,1)


fig['layout'].update(height=600, width=2000)
fig.show()

In [14]:
df=wqlib.wh_query_load(start_date=sdate_load,end_date=edate_load,wh_name=wh_name,delta=time_delta,n=threshold)
df.head()

,query_id,hourly_start_time,query_type,query_text,user_name,role_name,database_name,schema_name,warehouse_size,warehouse_type,bytes_scanned,percentage_scanned_from_cache,bytes_spilled_to_local_storage,bytes_spilled_to_remote_storage,partitions_total,total_time_elapsed_sec,compilation_time_sec,execution_time_sec,queued_provisioning_time_sec,queued_repair_time_sec,queued_overload_time_sec,list_external_files_time_sec,query_tag,execution_status
39761,01a78761-0604-ec7d-0000-08d15dd470a2,2022-10-10 06:00:00-07:00,SELECT,SELECT top_country FROM dbt_materialized.dbt_d...,VERTEX_API_PROD,VERTEX_API_PROD_ROLE,KIVA_PROD,<NA>,<NA>,STANDARD,0.0,0.0,0.0,0.0,0.0,0.04,0.04,0.00,0.0,0.0,0.0,0.00,,SUCCESS
39762,01a7875c-0604-ec7d-0000-08d15dd45ed6,2022-10-10 06:00:00-07:00,MERGE,"MERGE INTO ""SNOWPLOW"".""FIVETRAN_AUDIT"" AS ""MAI...",FIVETRAN_USER,FIVETRAN_ROLE,KIVA_PROD,<NA>,Small,STANDARD,671731712.0,100.0,0.0,0.0,164212.0,6.87,1.78,5.09,0.0,0.0,0.0,0.00,,SUCCESS
39763,01a7874d-0604-f2a6-0000-08d15dd44ec2,2022-10-10 06:00:00-07:00,COPY,"COPY INTO ""FIVETRAN_KIVA_RULES_STAGING"".""KDB_D...",FIVETRAN_USER,FIVETRAN_ROLE,KIVA_PROD,<NA>,Small,STANDARD,0.0,0.0,0.0,0.0,0.0,1.26,0.10,0.63,0.0,0.0,0.0,0.53,,SUCCESS
39764,01a7874d-0604-ec7d-0000-08d15dd4525a,2022-10-10 06:00:00-07:00,SHOW,"SHOW COLUMNS IN ""SNOWPLOW"".""FIVETRAN_AUDIT""",FIVETRAN_USER,FIVETRAN_ROLE,KIVA_PROD,<NA>,<NA>,STANDARD,0.0,0.0,0.0,0.0,0.0,0.04,0.03,0.01,0.0,0.0,0.0,0.00,,SUCCESS
39765,01a7875f-0604-f2a6-0000-08d15dd46bb6,2022-10-10 06:00:00-07:00,SELECT,select max(percentile) as percentile fr...,VERTEX_API_PROD,VERTEX_API_PROD_ROLE,KIVA_PROD,<NA>,Small,STANDARD,37376.0,100.0,0.0,0.0,31.0,0.08,0.04,0.03,0.0,0.0,0.0,0.00,,SUCCESS
